In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd
import numpy as np

import datetime

from CoolProp.CoolProp import PropsSI
from math import exp, factorial, ceil

import matplotlib.pyplot as plt
%matplotlib inline

cwd = os.getcwd()
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'standalone')))

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv("out.csv", index_col=0)

In [ ]:
df.head(2)

In [ ]:
start_time = datetime.datetime(month=1, day=1, year=2018, hour=0, minute=0, second=0)

In [ ]:
l = df['Simulation Time'].tolist()

In [ ]:
dt = [datetime.timedelta(seconds=x) for x in l]

In [ ]:
df.set_index(pd.to_datetime([start_time + x for x in dt]), inplace=True)

In [ ]:
df.plot(y=['GLHE Inlet Temperature [C]', 'GLHE Outlet Temperature [C]'])

In [ ]:
dT = df['GLHE Inlet Temperature [C]'].diff()
dt = df['GLHE Inlet Temperature [C]'].index.to_series().diff().dt.total_seconds()

df['dT_in/dt'] = dT/dt

In [ ]:
df.plot(y='dT_in/dt')

In [ ]:
df = df.loc['01-01-2018 02:50:00':'01-01-2018 03:30:00']

In [ ]:
def hanby(time, vol_flow_rate, volume):
    """
    Computes the non-dimensional response of a fluid conduit
    assuming well mixed nodes. The model accounts for the thermal
    capacity of the fluid and diffusive mixing.

    Hanby, V.I., J.A. Wright, D.W. Fetcher, D.N.T. Jones. 2002
    'Modeling the dynamic response of conduits.' HVAC&R Research 8(1): 1-12.

    The model is non-dimensional, so input parameters should have consistent units
    for that are able to compute the non-dimensional time parameter, tau.

    :math \tau = \frac{\dot{V} \cdot t}{Vol}


    :param time: time of fluid response
    :param vol_flow_rate: volume flow rate
    :param volume: volume of fluid circuit
    :return:
    """

    tau = vol_flow_rate * time / volume
    num_nodes = 20
    ret_sum = 1
    for i in range(1, num_nodes):
        ret_sum += (num_nodes * tau) ** i / factorial(i)

    return 1 - exp(-num_nodes * tau) * ret_sum

In [ ]:
def hanby_c(time, vol_flow_rate, volume):
    return 1 - hanby(time, vol_flow_rate, volume)

In [ ]:
delta_t = df['Simulation Time'][1] - df['Simulation Time'][0]
flow = 0.0002
vol = 0.05688

In [ ]:
def calc_exft_correction_factors(timestep, flow_rate, volume):
    t_tr = volume / flow_rate
    time = np.arange(0, t_tr * 2, timestep)
    f = np.array([hanby(x, flow_rate, volume) for x in time])
    d = np.diff(f)
    r = np.diff(f) / sum(d)
#     r = np.append(np.zeros(ceil(t_tr/timestep)), r)
    if len(r) == 0:
        return np.ones(1)
    else:
        return r

In [ ]:
calc_exft_correction_factors(120, flow, vol)

In [ ]:
def update_exft_correction_factors(r):
    if len(r) == 1:
        return r
    elif r[0] == 1:
        return r
    else:
        pop_val = r[0]
        l = np.count_nonzero(r) - 1
        delta = pop_val / l
        for i, val in enumerate(r):
            if r[i] == 0:
                break
            else:
                r[i] += delta

        r = np.roll(r, -1)
        r[-1] = 0
        return r

In [ ]:
cf_0 = calc_exft_correction_factors(delta_t, flow, vol)
cf_0

In [ ]:
cf_1 = update_exft_correction_factors(cf_0)
cf_1

In [ ]:
cf_2 = update_exft_correction_factors(cf_1)
cf_2

In [ ]:
cf_3 = update_exft_correction_factors(cf_2)
cf_3

In [ ]:
cf_4 = update_exft_correction_factors(cf_3)
cf_4

In [ ]:
def calc_exft(signal, to_correct):

    r = calc_exft_correction_factors(delta_t, flow, vol)
#     r = np.array(l)
    
    prev_temps = np.ones(len(r)) * to_correct[0]
    prev_signal = signal[0]
    
    dT_dt_prev = 0
    
    new_temps = np.empty([0])
    
    for i, t_sig in enumerate(signal):
        dT_dt = (t_sig - prev_signal) / delta_t
#         print(dT_dt, t_sig, prev_signal)
        
        if abs(dT_dt - dT_dt_prev) > 0.01:
            r = calc_exft_correction_factors(delta_t, flow, vol)
#             r = np.array(l)
        
        print(r)
        
        prev_temps[0] = to_correct[i]
        
        new_temp = sum(r * prev_temps)
#         print(to_correct[i], new_temp)
    
        new_temps = np.append(new_temps, new_temp)
#         print(new_temps)
        
        prev_temps = np.roll(prev_temps, 1)
        prev_temps[0] = new_temp
        
        r = update_exft_correction_factors(r)
        prev_sig = t_sig
        dT_dt_prev = dT_dt
        
#         if i == 10:
#             break
#         else:
#             print('\n')
    return new_temps

In [ ]:
t_c = calc_exft(df['GLHE Inlet Temperature [C]'], df['GLHE Outlet Temperature [C]'])
df['Corrected Temps'] = t_c

In [ ]:
df.plot(y=['GLHE Inlet Temperature [C]', 'GLHE Outlet Temperature [C]', 'Corrected Temps', 'Average Fluid Temp [C]'], marker='X')

In [ ]:
df.head(20)